In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-community langchain-openai langchain-ollama langchain-experimental langchainhub neo4j tiktoken yfiles_jupyter_graphspython-dotenv json-repair


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement yfiles_jupyter_graphspython-dotenv (from versions: none)
ERROR: No matching distribution found for yfiles_jupyter_graphspython-dotenv


In [4]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama


# Strict atomic knowledge schema
allowed_nodes = ["Person", "Attribute", "Location", "Event", "Organization", "Object", "Legend", "Entity"]
allowed_relationships = [
    ("Person", "has_attribute", "Attribute"),
    ("Person", "related_to", "Person"),
    ("Person", "lives_in", "Location"),
    ("Person", "works_at", "Organization"),
    ("Person", "interacts_with", "Person"),
    ("Person", "participated_in", "Event"),
    ("Person", "discovered", "Object"),
    ("Object", "linked_to", "Legend"),
    ("Location", "contains", "Object"),
    ("Legend", "mentions", "Entity"),
    ("Entity", "is_part_of", "Event"),
    ("Organization", "operates_in", "Location")
]


In [6]:
template = """
Extract a structured knowledge graph from this narrative. 
Use ONLY these node types: Person, Attribute, Location, Event, Organization, Object, Legend, Entity.
Use ONLY these relationships:
- Person → has_attribute → Attribute
- Person → related_to → Person
- Person → lives_in → Location
- Person → works_at → Organization
- Person → interacts_with → Person
- Person → participated_in → Event
- Person → discovered → Object
- Object → linked_to → Legend
- Location → contains → Object
- Legend → mentions → Entity
- Entity → is_part_of → Event
- Organization → operates_in → Location

Rules:
- Each node must contain only one atomic fact.
- Do NOT include full paragraphs or sentences in nodes.
- Prefer people, their roles, traits, and what they interact with.

Text:
{text}

Return JSON with "nodes" and "edges".
"""
prompt = PromptTemplate(input_variables=["text"], template=template)
chain = LLMChain(
    llm = Ollama(model="deepseek-r1:8b", temperature=0),
    prompt=prompt
)


C:\Users\jhay\AppData\Local\Temp\ipykernel_26688\3956208501.py:30: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:8b", temperature=0),
C:\Users\jhay\AppData\Local\Temp\ipykernel_26688\3956208501.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [8]:
llm = Ollama(model="deepseek-r1:8b", temperature=0)

llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes,
    allowed_relationships=allowed_relationships,
    strict_mode=True,
    node_properties=False,
    relationship_properties=False
)


In [ ]:
from langchain_core.documents import Document

with open("dum.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Split by paragraph for finer processing
paragraphs = [Document(page_content=p.strip()) for p in text.split("\n\n") if p.strip()]

graph_documents = llm_transformer.convert_to_graph_documents(paragraphs)


In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="your_password",
    refresh_schema=False
)

for doc in graph_documents:
    graph.add_graph_document(doc)


In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_openai import ChatOpenAI

qa_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0),
    qa_llm=ChatOpenAI(temperature=0),
    graph=graph,
    validate_cypher=True,
    use_function_response=True
)

# Example question:
result = qa_chain.run("What did Eleanor discover in the library?")
print(result)
